In [1]:
import pandas as pd
import numpy as np
import glob
import re
import os
import json

In [2]:
directory = os.getenv('HOME') +'/aiffel/aiffelthon/'

In [3]:
os.listdir(directory)

['MODELtf_ckpts_gd12_r1',
 'reg_2',
 'meta',
 'w2v_meaning_jj.model',
 'past_files',
 'onnx',
 '.ipynb_checkpoints',
 '[라벨]제주도_학습용데이터_1.zip',
 'corpus_ko.txt',
 '[라벨]전라도_학습데이터_1.zip',
 'added',
 'final',
 '[라벨]경상도_학습데이터_1.zip',
 'reg_1',
 'reg_4',
 'spm',
 'datset_full_raw_needs_translation_0221_v2.csv',
 '[라벨]충청도_학습데이터_1.zip',
 'reg_5',
 'saturi_crawling_thesaurus_0223.csv',
 'final_dataset.zip',
 '[라벨]강원도_학습데이터_1.zip',
 'reg_3',
 'DATA',
 'corpus_en.txt']

1 제주도
2 전라도
3 경상도
4 충청도
5 강원도

In [4]:
regs = ['reg_1','reg_2','reg_3','reg_4','reg_5']

In [6]:
json_files = glob.glob(directory +'reg_1/*.json')

In [7]:
import tqdm 
os.makedirs(directory +'meta', exist_ok=True)
def get_df(json_files,name):
#     df = pd.DataFrame(columns=['title', 'creator', 'distributor', 'year', 'category',
#        'annotation_level', 'sampling', 'author', 'publisher', 'date', 'topic',
#        'stdn','dial','eojeol_stdn'] )
    data = dict()
    data['reg'] = name
    data['data'] = []
    problem_file = set()
    with open(directory + f'meta/{name}.json', 'w') as js :
        for _,json_file in tqdm.tqdm(enumerate(json_files)) :
            with open(json_file, 'r', encoding='utf-8-sig') as f :
                try :
                    raw = json.load(f)
                except :
                    problem_file.add(json_file)
                    continue

            for i in range(len(raw['utterance'])) :
                match = re.search(r'\(\S+\)\/\(\S+\)', raw['utterance'][i]['form'])
                if match :
                    try :
                        new = raw['metadata'].copy()
                        new['age'] = raw['speaker'][0]['age']
                        new['sex'] = raw['speaker'][0]['sex']
                        new['prin_res'] = raw['speaker'][0]['principal_residence']                
                        new['curr_res'] = raw['speaker'][0]['current_residence']
                        try :
                            new['age2'] = raw['speaker'][1]['age']
                            new['sex2'] = raw['speaker'][1]['sex']
                            new['prin_res2'] = raw['speaker'][1]['principal_residence']                
                            new['curr_res2'] = raw['speaker'][1]['current_residence']
                        except IndexError:
                            new['age2'] = np.nan
                            new['sex2'] = np.nan
                            new['prin_res2'] = np.nan             
                            new['curr_res2'] = np.nan
                        new['stdn'] = raw['utterance'][i]['standard_form']
                        new['dial'] = raw['utterance'][i]['dialect_form']
                        form = raw['utterance'][i]['form']
                        new['eojeol_stdn'] = [re.findall(r'\(\S+\)\/\(\S+\)', form)]
                        data['data'].append(new)
                    except :
                        problem_file.add(json_file)
                        continue
                    
        json.dump(data,js)
        return problem_file

In [8]:
for reg in regs:
    json_files = glob.glob(directory + f'{reg}/*.json')
    print(len(json_files))
    get_df(json_files, reg)

5042


5042it [03:06, 27.08it/s]


7412


7412it [02:52, 42.86it/s]


7699


7699it [04:14, 30.29it/s]


6048


6048it [03:27, 29.21it/s]


4717


4717it [02:11, 35.77it/s]


In [11]:
test = glob.glob(directory + 'meta/*reg*.json')
df = pd.DataFrame(columns=['title', 'creator', 'distributor', 'year', 'category',
       'annotation_level', 'sampling', 'author', 'publisher', 'date', 'topic',
       'age', 'sex', 'prin_res', 'curr_res', 'age2', 'sex2', 'prin_res2',
       'curr_res2', 'stdn', 'dial', 'eojeol_stdn','reg'])

In [16]:
for i in test :
    with open(i,'r') as f :
        raw = json.load(f)
    temps = df_nested_list = pd.json_normalize(raw['data'])
    temps['reg'] = i[31:36]
    df = pd.concat([df,temps],axis=0)

In [13]:
i

'/aiffel/aiffel/aiffelthon/meta/reg_1.json'

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2996243 entries, 0 to 297601
Data columns (total 23 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   title             object
 1   creator           object
 2   distributor       object
 3   year              object
 4   category          object
 5   annotation_level  object
 6   sampling          object
 7   author            object
 8   publisher         object
 9   date              object
 10  topic             object
 11  age               object
 12  sex               object
 13  prin_res          object
 14  curr_res          object
 15  age2              object
 16  sex2              object
 17  prin_res2         object
 18  curr_res2         object
 19  stdn              object
 20  dial              object
 21  eojeol_stdn       object
 22  reg               object
dtypes: object(23)
memory usage: 548.6+ MB


In [18]:
mapper = {'reg_1':'jj','reg_2':'jd','reg_3':'gs','reg_4':'cc','reg_5':'kw'}
df['reg']= df['reg'].map(mapper)

In [19]:
df['reg'].value_counts()

jj    1173544
kw     915196
gs     334352
jd     297602
cc     275549
Name: reg, dtype: int64

In [21]:
df.to_csv(directory + 'meta/meta_data_included_raw_v2.csv', index=False)